# Introduction

> The script automates gathering of tweet data from the archives

> https://zhuanlan.zhihu.com/p/342441381

## Packages

In [30]:
#import packages 
import pandas as pd 
import numpy as np
import os 
import time
import requests
import itertools    
import json
import re

from datetime import date
from datetime import datetime
from datetime import timedelta

#import data scraping packages 
import tweepy
# from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from tweepy import Stream
from tweepy import Cursor
from tweepy import API

#import web scrapping packages 
import bs4 as bs
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import chromedriver_autoinstaller
from selenium_stealth import stealth
import undetected_chromedriver as uc



#sns module 
import snscrape.modules.twitter as sntwitter
import snscrape.modules.facebook as snfacebook
import snscrape.modules.instagram as sninstagram

#postgresql database packages 
import psycopg2
import psycopg2.extras as extras
from sqlalchemy import create_engine

import csv
from io import StringIO

#check the start time
start_time = datetime.now()

## Extract Time

In [31]:
#generate the 
number_of_dates = 365*0.5
end_date = date.today()
start_date =  end_date - timedelta(days=number_of_dates)
duration = end_date - start_date

print()
print()
print(f"The start date: {start_date} end date is: {end_date} and time duration is: {duration}")



The start date: 2022-02-23 end date is: 2022-08-24 and time duration is: 182 days, 0:00:00


In [3]:
#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%B %d %Y %H %M %S")

#print date 
print()
print()
print("Todays date: ", formated_date)



Todays date:  August 23 2022 20 37 57


### Keywords

> The following are search keywords for cyber security concerns 

In [33]:
nigerian_slag = ['gbege','trabaye','gbemi landicap',' our cousins',' yarn opata',' dey don jonz me',' i don loose guard',
                 ' won ti bo card','maga don pay',' money miss road',' mon te ke',' mumu me',' e don enter',' won ti gbami',
                 ' won gba mi loju',' abaye','gbemi landicap',' our cousins',' yarn opata',' dey don jonz me',
                 ' i don loose guard',' money miss road',' mon te ke',' mumu me',' e don enter',' won ti gbami',
                 ' won gba mi loju',' yawa don gas',' normal level',' that format',' sinzu money',' aza',' bad market',
                 'maga ti sanwo',' maaye ti gbowo wole',' osu ti sanwo',' client ti gbemi landicap',' mo fe trabaye bayi',
                 'wired my moni',' don comot my money o',' e don shele',' yawa don gash',' gbege don start',' see gbege',
                 ' see kasala',' wetin dey sub',' which wayyyy',' mugu don pay',' kasala don burst!','moti wo gau',
                 ' mewa nsele',' owo jona',' owo wogbo',' ole gbe',' kele gbe',' won semi ni janba',' won fun mi ni format',
                 '419',' wuruwuru',' gbajue',' one chance',' wonti gba',' won lu ni jibiti',' dey do mi wayo',' magomago',
                 ' ole',' duped',' snatched',' skimming',' farb',' jaguda',' gbewiri',' dey don run streets',
                 ' e reason me bad',' dey don use am for me',' dey don do me ajao.','don hack my account',' dey don kpokpo me',
                 ' dey dagbo','dey don carry me',' peak me',' wire']

#fornat the sland 
print()
print()
print(f"{'*'*50} Keyword list {'*'*60}")
nigerian_slag = [x.strip().lower() for x in nigerian_slag]
print(nigerian_slag)

print()
print()

#search words 
print(f"{'*'*50} Joined Keyword List {'*'*50}")
search_word = "|".join([j.lower() for j in nigerian_slag])
print(search_word)





************************************************** Keyword list ************************************************************
['gbege', 'trabaye', 'gbemi landicap', 'our cousins', 'yarn opata', 'dey don jonz me', 'i don loose guard', 'won ti bo card', 'maga don pay', 'money miss road', 'mon te ke', 'mumu me', 'e don enter', 'won ti gbami', 'won gba mi loju', 'abaye', 'gbemi landicap', 'our cousins', 'yarn opata', 'dey don jonz me', 'i don loose guard', 'money miss road', 'mon te ke', 'mumu me', 'e don enter', 'won ti gbami', 'won gba mi loju', 'yawa don gas', 'normal level', 'that format', 'sinzu money', 'aza', 'bad market', 'maga ti sanwo', 'maaye ti gbowo wole', 'osu ti sanwo', 'client ti gbemi landicap', 'mo fe trabaye bayi', 'wired my moni', 'don comot my money o', 'e don shele', 'yawa don gash', 'gbege don start', 'see gbege', 'see kasala', 'wetin dey sub', 'which wayyyy', 'mugu don pay', 'kasala don burst!', 'moti wo gau', 'mewa nsele', 'owo jona', 'owo wogbo', 'ole gbe', 'kele 

### African Countries

In [5]:
#countries involved
name = ['Algeria','Angola','Benin','Botswana','Burkina_Faso','Burundi','Cameroon','Cape_Verde',
             'Central_African_Republic','Chad','Comoros','Democratic_Republic_of_the_Congo','Djibouti','Egypt',
             'Equatorial_Guinea',' Eritrea',' Ethiopia',' Gabon',' Gambia',' Ghana','Guinea',' Guinea-Bissau',
             'Ivory_Coast','Kenya',' Lesotho',' Liberia',' Libya',' Madagascar',' Malawi',' Mali',' Mauritania','Mauritius',
             'Morocco',' Mozambique',' Namibia',' Niger',' Nigeria','Republic of the Congo',' Rwanda','Sao_Tome_and_Principe',
             'Senegal',' Seychelles',' Sierra_Leone',' Somalia',' South_Africa',' Sudan','South_Sudan','Swaziland',' Tanzania',' Togo',
             'Tunisia',' Uganda',' Zambia',' Zimbabwe']

# #strip any white spaces 
country_name = [x.strip().lower().replace('_','-').replace(' ','-') for x in name]

#search words 
countries = "|".join([j.strip().lower() for j in name])

In [6]:
options = webdriver.ChromeOptions()
options.add_argument("start-maximized")
options.add_argument("--headless")
options.add_experimental_option("excludeSwitches", ["enable-automation"])
options.add_experimental_option('useAutomationExtension', False)

In [7]:
 driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')

C:\Users\moses\AppData\Local\Temp\ipykernel_7584\1516475625.py:1: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(r'C:\Users\chromedriver_win32\chromedriver.exe')


## Place Specific 

> Countries towns and cities co-ordinates [link](https://www.countrycoordinate.com/)

In [8]:
#get the data in a dataframe 
coordinates_df =  pd.DataFrame()
count = 0

for country in country_name:
    #get the url
    url = f"https://www.countrycoordinate.com/country-{country}/"
    count +=1
    print(f"Num: {count}; link: {url}") 
    try:
        driver = uc.Chrome(use_subprocess=True)
        driver.get(url)
        time.sleep(5)
        soup = bs.BeautifulSoup(driver.page_source,'lxml')

        # get data table & save as dataframe
        table_val = soup.find("table",{'table table-bordered table-hover'})
        table_val_df = pd.read_html(str(table_val))[0]

        table_val_df.columns = ['city', 'coords']
        table_val_df['country'] = country
        table_val_df['url'] = url
        table_val_df[['lat', 'lon']] = table_val_df['coords'].str.split('/', expand = True)
        table_val_df.drop(columns = ['coords'], inplace = True)
        if table_val_df.shape[0]>1:
            coordinates_df = pd.concat([coordinates_df,table_val_df])
        else:
            pass


        #get the number of other pages 
        page_num_table = soup.find_all("table", {"class":"table table-bordered centered"})



          #loop through other pages
        print(f"{'*'*15} base page done {'*'*15}")
        print(f"{'*'*15} Other pages {'*'*15}")
        for tab in page_num_table:
            row = tab.find_all('a')
            row = [i.text.strip() for i in row]
            row = row[:-1]
            print(f"{'*'*15} pages to extract {len(row)+1} {'*'*15}")

            for page in row: 
                next_page_url = url+f"?page={page}"
                print(next_page_url)

                #get the data 
                driver.get(next_page_url)
                time.sleep(2)
                soup_page = bs.BeautifulSoup(driver.page_source, 'lxml')

                # get data table & save as dataframe
                table_val_next = soup_page.find("table",{'table table-bordered table-hover'})
                table_val_df_1 = pd.read_html(str(table_val_next))[0]
                table_val_df_1.columns = ['city', 'coords']
                table_val_df_1['country'] = country
                table_val_df_1['url'] = url
                table_val_df_1[['lat', 'lon']] = table_val_df_1['coords'].str.split('/', expand = True)
                table_val_df_1.drop(columns = ['coords'], inplace = True)
                if table_val_df_1.shape[0]>1:
                    coordinates_df = pd.concat([coordinates_df,table_val_df_1])
                else:
                    pass
                
    except Exception as e:
        print(f'The error is: {str(e)}')
        pass
    
#clean the data 

#reset the index
coordinates_df.reset_index(drop = True, inplace = True)

#clean the data 
coordinates_df['city'] = coordinates_df['city'].str.split('(', expand = True)[0]

#drop duplicate towns
print(f"Rows befor droping duplicates {coordinates_df.shape[0]}")
coordinates_df.drop_duplicates(subset = ['city', 'country'], inplace = True, keep = 'last')
print(f"Rows befor droping duplicates {coordinates_df.shape[0]}")

#print the sample 
coordinates_df.sample(3)

Num: 1; link: https://www.countrycoordinate.com/country-algeria/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 7 ***************
https://www.countrycoordinate.com/country-algeria/?page=2
https://www.countrycoordinate.com/country-algeria/?page=3
https://www.countrycoordinate.com/country-algeria/?page=4
https://www.countrycoordinate.com/country-algeria/?page=5
https://www.countrycoordinate.com/country-algeria/?page=12
https://www.countrycoordinate.com/country-algeria/?page=13
Num: 2; link: https://www.countrycoordinate.com/country-angola/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoordinate.com/country-angola/?page=2
Num: 3; link: https://www.countrycoordinate.com/country-benin/
*************** base page done ***************
*************** Other pages ***************
*************** pages to e

*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/country-libya/?page=2
https://www.countrycoordinate.com/country-libya/?page=3
Num: 28; link: https://www.countrycoordinate.com/country-madagascar/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 6 ***************
https://www.countrycoordinate.com/country-madagascar/?page=2
https://www.countrycoordinate.com/country-madagascar/?page=3
https://www.countrycoordinate.com/country-madagascar/?page=4
https://www.countrycoordinate.com/country-madagascar/?page=5
https://www.countrycoordinate.com/country-madagascar/?page=6
Num: 29; link: https://www.countrycoordinate.com/country-malawi/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 2 ***************
https://www.countrycoor

Num: 52; link: https://www.countrycoordinate.com/country-uganda/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-uganda/?page=2
https://www.countrycoordinate.com/country-uganda/?page=3
https://www.countrycoordinate.com/country-uganda/?page=4
Num: 53; link: https://www.countrycoordinate.com/country-zambia/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 4 ***************
https://www.countrycoordinate.com/country-zambia/?page=2
https://www.countrycoordinate.com/country-zambia/?page=3
https://www.countrycoordinate.com/country-zambia/?page=4
Num: 54; link: https://www.countrycoordinate.com/country-zimbabwe/
*************** base page done ***************
*************** Other pages ***************
*************** pages to extract 3 ***************
https://www.countrycoordinate.com/c

,city,country,url,lat,lon
1375,Nandihizana,madagascar,https://www.countrycoordinate.com/country-mada...,-20.834543,47.118940
2349,Mkwanga,tanzania,https://www.countrycoordinate.com/country-tanz...,-3.808384,37.723768
444,Ponta do Sol,cape-verde,https://www.countrycoordinate.com/country-cape...,17.200483,-25.092562


> Select Nigeria 

In [9]:
nigeria = coordinates_df[coordinates_df['country'] == 'nigeria']
nigeria.reset_index(drop = True, inplace = True)
# coordinates_df
nigeria

,city,country,url,lat,lon
0,Lagos,nigeria,https://www.countrycoordinate.com/country-nige...,6.524379,3.379206
1,Kano,nigeria,https://www.countrycoordinate.com/country-nige...,12.002179,8.591956
2,Ibadan,nigeria,https://www.countrycoordinate.com/country-nige...,7.377535,3.947040
3,Kaduna,nigeria,https://www.countrycoordinate.com/country-nige...,10.510464,7.416505
4,Port Harcourt,nigeria,https://www.countrycoordinate.com/country-nige...,4.815554,7.049844
...,...,...,...,...,...
122,Obolo-Eke,nigeria,https://www.countrycoordinate.com/country-nige...,6.874221,7.617496
123,Disina,nigeria,https://www.countrycoordinate.com/country-nige...,11.481895,9.917174
124,Bornu Yassu,nigeria,https://www.countrycoordinate.com/country-nige...,12.271107,12.580724
125,Ijebu-Ife,nigeria,https://www.countrycoordinate.com/country-nige...,6.780910,4.033785


> Extract tweets with keywords

In [10]:
#create and empty df
df_x = pd.DataFrame()

#define the max keywords
maxcounts = 50
count = 0

try: 
    for row in nigeria.index:
            country = nigeria['country'][row]
            nearby_location = nigeria['city'][row]
            lat = nigeria['lat'][row]
            lon = nigeria['lon'][row]
            circumfrence = "20km"
            coor = lat+","+lon+","+circumfrence
            coor = format(coor)
            print()
            print(f"country: {country}, nearby_loc: {nearby_location},  coordinates: {lat+', '+lon}, radius: {circumfrence}")
            
            try:
                for word in nigerian_slag:
                    count +=1
                    
                    print(f"{'*'*20} num: {count};  word: {word} {'*'*20}")
                    scraped_tweets = sntwitter.TwitterSearchScraper(f'{str(word)} since:{start_date} until:{end_date} geocode:"{coor}"').get_items()

                    sliced_scraped_tweets = itertools.islice(scraped_tweets, maxcounts)  
                    df_coord = pd.DataFrame(sliced_scraped_tweets)
                    df_coord.columns = df_coord.columns.str.lower()
                    df_coord['keyword'] = word
                    df_coord['nearby_location'] = nearby_location
                    df_coord['country'] = country
                    df_x = pd.concat([df_x, df_coord])
            except Exception as e:
                print(f'The error was: {str(e)}')
except Exception as e:
    print(f'The error was: {str(e)}')


country: nigeria, nearby_loc: Lagos,  coordinates: 6.524379 ,  3.379206, radius: 20km
******************** num: 1;  word: gbege ********************
******************** num: 2;  word: trabaye ********************
******************** num: 3;  word: gbemi landicap ********************
******************** num: 4;  word: our cousins ********************
******************** num: 5;  word: yarn opata ********************
******************** num: 6;  word: dey don jonz me ********************
******************** num: 7;  word: i don loose guard ********************
******************** num: 8;  word: won ti bo card ********************
******************** num: 9;  word: maga don pay ********************
******************** num: 10;  word: money miss road ********************
******************** num: 11;  word: mon te ke ********************
******************** num: 12;  word: mumu me ********************
******************** num: 13;  word: e don enter ********************
********

******************** num: 115;  word: bad market ********************
******************** num: 116;  word: maga ti sanwo ********************
******************** num: 117;  word: maaye ti gbowo wole ********************
******************** num: 118;  word: osu ti sanwo ********************
******************** num: 119;  word: client ti gbemi landicap ********************
******************** num: 120;  word: mo fe trabaye bayi ********************
******************** num: 121;  word: wired my moni ********************
******************** num: 122;  word: don comot my money o ********************
******************** num: 123;  word: e don shele ********************
******************** num: 124;  word: yawa don gash ********************
******************** num: 125;  word: gbege don start ********************
******************** num: 126;  word: see gbege ********************
******************** num: 127;  word: see kasala ********************
******************** num: 128;  w

******************** num: 228;  word: dey do mi wayo ********************
******************** num: 229;  word: magomago ********************
******************** num: 230;  word: ole ********************
******************** num: 231;  word: duped ********************
******************** num: 232;  word: snatched ********************
******************** num: 233;  word: skimming ********************
******************** num: 234;  word: farb ********************
******************** num: 235;  word: jaguda ********************
******************** num: 236;  word: gbewiri ********************
******************** num: 237;  word: dey don run streets ********************
******************** num: 238;  word: e reason me bad ********************
******************** num: 239;  word: dey don use am for me ********************
******************** num: 240;  word: dey don do me ajao. ********************
******************** num: 241;  word: don hack my account ********************
****

******************** num: 340;  word: mumu me ********************
******************** num: 341;  word: e don enter ********************
******************** num: 342;  word: won ti gbami ********************
******************** num: 343;  word: won gba mi loju ********************
******************** num: 344;  word: abaye ********************
******************** num: 345;  word: gbemi landicap ********************
******************** num: 346;  word: our cousins ********************
******************** num: 347;  word: yarn opata ********************
******************** num: 348;  word: dey don jonz me ********************
******************** num: 349;  word: i don loose guard ********************
******************** num: 350;  word: money miss road ********************
******************** num: 351;  word: mon te ke ********************
******************** num: 352;  word: mumu me ********************
******************** num: 353;  word: e don enter ********************
*

******************** num: 453;  word: gbege don start ********************
******************** num: 454;  word: see gbege ********************
******************** num: 455;  word: see kasala ********************
******************** num: 456;  word: wetin dey sub ********************
******************** num: 457;  word: which wayyyy ********************
******************** num: 458;  word: mugu don pay ********************
******************** num: 459;  word: kasala don burst! ********************
******************** num: 460;  word: moti wo gau ********************
******************** num: 461;  word: mewa nsele ********************
******************** num: 462;  word: owo jona ********************
******************** num: 463;  word: owo wogbo ********************
******************** num: 464;  word: ole gbe ********************
******************** num: 465;  word: kele gbe ********************
******************** num: 466;  word: won semi ni janba ********************
**

...

******************** num: 7944;  word: gbewiri ********************
******************** num: 7945;  word: dey don run streets ********************
******************** num: 7946;  word: e reason me bad ********************
******************** num: 7947;  word: dey don use am for me ********************
******************** num: 7948;  word: dey don do me ajao. ********************
******************** num: 7949;  word: don hack my account ********************
******************** num: 7950;  word: dey don kpokpo me ********************
******************** num: 7951;  word: dey dagbo ********************
******************** num: 7952;  word: dey don carry me ********************
******************** num: 7953;  word: peak me ********************
******************** num: 7954;  word: wire ********************

country: nigeria, nearby_loc: Igede-Ekiti,  coordinates: 7.670522 ,  5.124966, radius: 20km
******************** num: 7955;  word: gbege ********************
*****************

******************** num: 8054;  word: our cousins ********************
******************** num: 8055;  word: yarn opata ********************
******************** num: 8056;  word: dey don jonz me ********************
******************** num: 8057;  word: i don loose guard ********************
******************** num: 8058;  word: money miss road ********************
******************** num: 8059;  word: mon te ke ********************
******************** num: 8060;  word: mumu me ********************
******************** num: 8061;  word: e don enter ********************
******************** num: 8062;  word: won ti gbami ********************
******************** num: 8063;  word: won gba mi loju ********************
******************** num: 8064;  word: yawa don gas ********************
******************** num: 8065;  word: normal level ********************
******************** num: 8066;  word: that format ********************
******************** num: 8067;  word: sinzu money

******************** num: 8166;  word: mugu don pay ********************
******************** num: 8167;  word: kasala don burst! ********************
******************** num: 8168;  word: moti wo gau ********************
******************** num: 8169;  word: mewa nsele ********************
******************** num: 8170;  word: owo jona ********************
******************** num: 8171;  word: owo wogbo ********************
******************** num: 8172;  word: ole gbe ********************
******************** num: 8173;  word: kele gbe ********************
******************** num: 8174;  word: won semi ni janba ********************
******************** num: 8175;  word: won fun mi ni format ********************
******************** num: 8176;  word: 419 ********************
******************** num: 8177;  word: wuruwuru ********************
******************** num: 8178;  word: gbajue ********************
******************** num: 8179;  word: one chance ********************


******************** num: 8278;  word: dey don kpokpo me ********************
******************** num: 8279;  word: dey dagbo ********************
******************** num: 8280;  word: dey don carry me ********************
******************** num: 8281;  word: peak me ********************
******************** num: 8282;  word: wire ********************

country: nigeria, nearby_loc: Abocho,  coordinates: 7.571645 ,  6.990720, radius: 20km
******************** num: 8283;  word: gbege ********************
******************** num: 8284;  word: trabaye ********************
******************** num: 8285;  word: gbemi landicap ********************
******************** num: 8286;  word: our cousins ********************
******************** num: 8287;  word: yarn opata ********************
******************** num: 8288;  word: dey don jonz me ********************
******************** num: 8289;  word: i don loose guard ********************
******************** num: 8290;  word: won ti bo

KeyboardInterrupt: 

In [12]:
df_xy = df_x.copy()

In [40]:
spyware = 100
# ramsomware = 100
lagos = 1


output = spyware*lagos
print(output)

# len(nigerian_slag)

100


In [ ]:
slag = ''


> Clean the extracted tweets

In [13]:

#subset the data
df = df_x[['id', 'date', 'content', 'renderedcontent', 'user', 'replycount','retweetcount', 'likecount', 'quotecount',
           'url', 'lang', 'retweetedtweet', 'quotedtweet','mentionedusers', 'coordinates', 'place', 'hashtags', 'keyword', 
           'nearby_location', 'country']]

#format the date
df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))

# getting user's location
df.reset_index(drop = True, inplace = True)
df['username'] = df['user'].apply(lambda x: x['username'])
df['displayname'] = df['user'].apply(lambda x: x['displayname'])
df['user_id'] = df['user'].apply(lambda x: x['id'])
df['user_location'] = df['user'].apply(lambda x: x['location'])
df['description'] = df['user'].apply(lambda x: x['description'])
df['followersCount'] = df['user'].apply(lambda x: x['followersCount'])
df['friendsCount'] = df['user'].apply(lambda x: x['friendsCount'])
df['statusesCount'] = df['user'].apply(lambda x: x['statusesCount'])
df['favouritesCount'] = df['user'].apply(lambda x: x['favouritesCount'])
df['protected'] = df['user'].apply(lambda x: x['protected'])

#mentions 
pattern_1 = r'#(\w+)'
pattern_2 = r'@(\w+)'
# df['hashtag'] = df['content'].apply(lambda x: re.findall(pattern_1, x))
df['mentions'] = df['content'].apply(lambda x: re.findall(pattern_2, x))

#drop some columns 
df.drop(columns = ['user', 'mentionedusers'], inplace = True)


#clean the data 
##lower the text
df['content'] = df['content'].apply(lambda x: x.lower())
df['content'] = df['content'].apply(lambda x: x.strip())
df['content'] = df['content'].apply(lambda x: x.replace("  ", " "))
df['content'] = df['content'].apply(lambda x: x.replace("\n", ""))

##strip white spaces 
df.columns = df.columns.str.strip()

#lower all the names 
df.colums = df.columns.str.lower()


#filter words that contained in the list 
df['search_word'] = np.where(df['content'].str.contains(search_word),'Yes','No')

##drop duplicate tweets if any 
print(f'The number of tweets before droping duplicates are: {df.shape[0]}')
df = df.drop_duplicates(subset=['date', 'id', 'content', 'username', 'user_location'], keep= 'last')
print(f'The number of tweets after droping duplicates are: {df.shape[0]}')

#print the data
df.sample(5)


C:\Users\moses\AppData\Local\Temp\ipykernel_7584\3658505057.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['date'] = df['date'].apply(lambda x: x.strftime("%d-%b-%Y %H:%M:%S"))
C:\Users\moses\AppData\Local\Temp\ipykernel_7584\3658505057.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['username'] = df['user'].apply(lambda x: x['username'])
C:\Users\moses\AppData\Local\Temp\ipykernel_7584\3658505057.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

The number of tweets before droping duplicates are: 37764
The number of tweets after droping duplicates are: 18297


,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
3972,1.560884e+18,20-Aug-2022 06:56:40,naso una dey take start una agenda… unfortunat...,Naso una dey take start una agenda… unfortunat...,0.0,0.0,0.0,0.0,https://twitter.com/iamsardeeq/status/15608835...,en,...,793917370096095232,"Abuja, Nigeria",Media and Communication Expert || Consultant |...,6395,5220,48139,96399,False,[],No
35684,1.560616e+18,19-Aug-2022 13:14:46,@tumza_mellowhun @robinkhay never acted on his...,@Tumza_mellowhun @robinkhay Never acted on his...,2.0,0.0,0.0,0.0,https://twitter.com/TheEagle2009/status/156061...,en,...,400633072,"Lagos,Nigeria",Conversations about @ozo_chukwu plus all thing...,147,52,11792,19065,False,"[Tumza_mellowhun, robinkhay]",Yes
32170,1.560547e+18,19-Aug-2022 08:38:36,@qj_wire @joshuaperegbe @badmanstephen @obacru...,@QJ_wire @JoshuaPeregbe @badmanstephen @Obacru...,1.0,0.0,1.0,0.0,https://twitter.com/Leeeh_Faaaaah/status/15605...,en,...,1432175482578014210,Somewhere In Your Heart🤍,Good Good Life No Stress 😊🌬❤️,242,168,1656,11346,False,"[QJ_wire, JoshuaPeregbe, badmanstephen, Obacruze]",Yes
6196,1.194248e+18,12-Nov-2019 13:39:49,so aisha buhari organises a maulud at villa an...,So Aisha Buhari organises a Maulud at Villa an...,0.0,2.0,3.0,0.0,https://twitter.com/AdamsybassiMoha/status/119...,en,...,1105538186204405760,"Maiduguri, Nigeria",vibrant young leader,1583,4878,1020,672,False,[],No
33572,1.103039e+18,05-Mar-2019 21:06:50,@akin_makinde a good or bad market ọga adé? ht...,@akin_makinde a good or bad market ọga adé? tw...,1.0,0.0,0.0,0.0,https://twitter.com/konquerorAbdul/status/1103...,hu,...,289342150,"Abuja, Nigeria",#MuslimStriving4Jannah\n#Husband\n#Father\n#Un...,378,457,9270,6353,False,[akin_makinde],Yes


> Tweets containing keywords

In [14]:
df['search_word'].value_counts(normalize = True).mul(100).round(1).astype(str) + '%'

Yes    75.3%
No     24.7%
Name: search_word, dtype: object

> Drop the tweets that do not have the keywords

In [15]:
#filter out the only data with matching keyword
data = df[df['search_word']!='No']
data.reset_index(drop = True, inplace = True)
data.head(5)

,id,date,content,renderedcontent,replycount,retweetcount,likecount,quotecount,url,lang,...,user_id,user_location,description,followersCount,friendsCount,statusesCount,favouritesCount,protected,mentions,search_word
0,1.345730e+18,03-Jan-2021 13:52:03,it's not even 4k ? money miss road then 😭 http...,It's not even 4k ? Money miss road then 😭 twit...,0.0,0.0,0.0,0.0,https://twitter.com/d0zie/status/1345729641257...,en,...,40921635,The Bridge,,1529,702,117489,3279,False,[],Yes
1,1.344031e+18,29-Dec-2020 21:20:20,money miss road!,Money miss road!,0.0,0.0,2.0,0.0,https://twitter.com/TemmyDiran/status/13440305...,en,...,751095566919237633,Ikota,Certified Historian | Passionate Logistics Exp...,2776,3080,55336,2700,False,[],Yes
2,1.333378e+18,30-Nov-2020 11:49:14,see my generation of money miss road.god have ...,See my generation of money miss road.\nGod hav...,0.0,0.0,0.0,0.0,https://twitter.com/Bodii06/status/13333775471...,en,...,256500380,"lagos,Nigeria","A friend,a brother, a father💞,cool n calm guy,...",557,1262,34033,20621,False,[],Yes
3,1.327527e+18,14-Nov-2020 08:20:22,this na money miss road walahi. no be everybod...,This Na money miss road walahi. No be everybod...,0.0,0.0,0.0,0.0,https://twitter.com/flameboxx/status/132752677...,en,...,2578659616,,2 Cor 9:8. LFC. Travis$. Giannis. Ambivert 🔛,1535,1298,7656,11674,False,[],Yes
4,1.309893e+18,26-Sep-2020 16:29:57,@havertzinho29 @truecfc_ @sz_afc @guendouzxv2 ...,@Havertzinho29 @TrueCFC_ @SZ_AFC @guendouzxV2 ...,0.0,0.0,0.0,0.0,https://twitter.com/kris4tune/status/130989297...,en,...,78268801,"Lagos, Nigeria",Jesus Christ is Lord of my life. I'm a Brand ...,1750,4847,26828,7516,False,"[Havertzinho29, TrueCFC_, SZ_AFC, guendouzxV2]",Yes


## Save the File

> Create file Name

In [16]:
#generate a filename 
name = f"nigerian_slang_tweets"

#get todays date
today = datetime.now()

# format the dates
formated_date = today.strftime("%d_%b").lower()

#extension
extension = ".xlsx"

#generate the name
new_filename = name+"_"+formated_date
old_filename = name+"_"+formated_date+"_"+"oldfile"
print(old_filename ,"\n", new_filename)

nigerian_slang_tweets_24_aug_oldfile 
 nigerian_slang_tweets_24_aug


> Write data to Postgresql

In [29]:
#create the connection 

db_host = 'research-dev-db.cluster-c30rc84jwwhk.us-east-2.rds.amazonaws.com'
# db_host = '192.168.8.221'
db_port = 5432
db_password = 'researchpassword'
db = 'postgres'
db_user = 'postgres'


def psql_insert_copy(table, conn, keys, data_iter):
    # gets a DBAPI connection that can provide a cursor
    dbapi_conn = conn.connection
    with dbapi_conn.cursor() as cur:
        s_buf = StringIO()
        writer = csv.writer(s_buf)
        writer.writerows(data_iter)
        s_buf.seek(0)

        columns = ', '.join('"{}"'.format(k) for k in keys)
        if table.schema:
            table_name = '{}.{}'.format(table.schema, table.name)
        else:
            table_name = table.name

        sql = 'COPY {} ({}) FROM STDIN WITH CSV'.format(
            table_name, columns)
        cur.copy_expert(sql=sql, file=s_buf)

engine = create_engine(f'postgresql://{db_user}:{db_password}@{db_host}:{db_port}/{db}')

#write the data if the table exists , replace the data
try:
    data.to_sql(new_filename, engine, method=psql_insert_copy, if_exists='replace')
except Exception as e:
    print(f'The error is: {str(e)}')
    pass

> write the african countries cities

In [ ]:
# #write the data if the table exists , replace the data
# table_name = "african_countries_towns_and_cities"

# #write the data into db
# try:
#     coordinates_df.to_sql(table_name, engine, method=psql_insert_copy, if_exists='replace')
# except Exception as e:
#     print(f'The error is: {str(e)}')
#     pass

> Rename file if present in the directory 

In [21]:
#rename the file if is in directory 
# old_filename+'.xlsx'
# new_filename+'xlsx'
try:
  os.rename(old_filename+'.xlsx', new_filename+'xlsx')
except Exception as e:
  print(f"The error is: {str(e)}")
  pass

The error is: [WinError 2] The system cannot find the file specified: 'nigerian_slang_tweets_24_aug_oldfile.xlsx' -> 'nigerian_slang_tweets_24_augxlsx'


> Save the File

In [23]:
#write the file
try:
    data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
except BaseException as e:
    print('-'*20)
    print(f"The error is {str(e)}")
    pass

In [ ]:
# #write the file
# try:
# #     data.to_excel(new_filename+'.xlsx', sheet_name = 'tweets data', index = False)
#     coordinates_df.to_excel("african cities coordinates"+'.xlsx', sheet_name = 'tweets data', index = False)
# except BaseException as e:
#     print('-'*20)
#     print(f"The error is {str(e)}")
#     pass

## Time Spend

In [24]:
#get the time used to proccess the data
end_time = datetime.now()
duration = end_time - start_time
print(f"The stopped at: {end_time} '\n' Started at: {end_time} '\n' and took {duration} to run")

The stopped at: 2022-08-24 08:19:49.042228 '
' Started at: 2022-08-24 08:19:49.042228 '
' and took 0:01:18.281891 to run
